<a href="https://colab.research.google.com/github/HemantajayW/Stock_market_prediction_during_WARPERIOD/blob/main/ukraine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

Loading main data

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
data = yf.download("BG" , start = "2018-01-01" , interval = '1d')

[*********************100%***********************]  1 of 1 completed


In [ ]:
data.head(3)
data.index=data.index.strftime('%Y-%m-%d')


In [ ]:
data.sort_index(inplace = True)

In [ ]:
data = data.loc[~data.index.duplicated(keep='first')]

In [ ]:
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-05-10,108.739998,111.099998,108.300003,110.330002,110.330002,1703000
2022-05-11,110.500000,113.010002,109.080002,109.769997,109.769997,1909900
2022-05-12,109.620003,110.514999,106.589996,108.650002,108.650002,710970


In [ ]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,67.230003,68.029999,66.449997,67.989998,59.197887,944500
2018-01-03,68.379997,70.209999,68.059998,69.449997,60.469097,2124600
2018-01-04,69.779999,71.209999,69.750000,70.470001,61.357189,2026900
2018-01-05,70.500000,70.989998,69.550003,69.940002,60.895721,795500
2018-01-08,69.750000,69.769997,68.239998,68.930000,60.016331,1041200


In [ ]:
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1.099000e+03
mean,66.680118,67.505200,65.798444,66.654113,62.876990,1.360471e+06
std,18.631073,18.845220,18.393017,18.639382,19.521213,7.477176e+05
min,30.000000,33.500000,29.000000,30.250000,28.398466,1.777000e+05
25%,53.435001,54.025000,52.945000,53.439999,48.976200,9.017000e+05
50%,64.000000,64.559998,63.259998,63.880001,57.982380,1.176900e+06
75%,77.945000,78.919998,76.794998,77.760002,75.900669,1.616900e+06
max,127.349998,128.399994,123.510002,126.760002,126.760002,8.434300e+06


Understanding Trends with in the Data

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['High'] , mode = 'lines',name='High'))
fig.add_trace(go.Scatter(x = data.index , y = data['Low'] , mode = 'lines',name='Low'))
fig.add_trace(go.Scatter(x = data.index , y = data['Open'] , mode = 'lines',name='Open'))
fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines',name='Close'))




fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Price')
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Volume'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Volume')
fig.show()

Data Preparation

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
import pickle 
from tqdm.notebook import tnrange

In [ ]:
# Filter only required data 
data = data[['Close' , 'Volume']]
data.head(3)
data.index

Index(['2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05', '2018-01-08',
       '2018-01-09', '2018-01-10', '2018-01-11', '2018-01-12', '2018-01-16',
       ...
       '2022-04-29', '2022-05-02', '2022-05-03', '2022-05-04', '2022-05-05',
       '2022-05-06', '2022-05-09', '2022-05-10', '2022-05-11', '2022-05-12'],
      dtype='object', name='Date', length=1099)

Scrapping extra information

In [ ]:
import requests 

response = requests.get('https://www.alphavantage.co/query?function=RSI&symbol=BG&interval=daily&time_period=5&series_type=close&apikey=43T9T17VCV2ME4SM') 
response = response.json()

In [ ]:
response.keys()

dict_keys(['Meta Data', 'Technical Analysis: RSI'])

In [ ]:
rsi_data = pd.DataFrame.from_dict(response['Technical Analysis: RSI'] , orient='index')

In [ ]:
rsi_data.head()

,RSI
2001-08-09,0.0000
2001-08-10,0.0000
2001-08-13,0.0000
2001-08-14,0.0000
2001-08-15,0.0000


In [ ]:
rsi_data = rsi_data[rsi_data.index >= '2018-01-01']
rsi_data.index

Index(['2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05', '2018-01-08',
       '2018-01-09', '2018-01-10', '2018-01-11', '2018-01-12', '2018-01-16',
       ...
       '2022-04-28', '2022-04-29', '2022-05-02', '2022-05-03', '2022-05-04',
       '2022-05-05', '2022-05-06', '2022-05-09', '2022-05-10', '2022-05-11'],
      dtype='object', length=1098)

In [ ]:
rsi_data['RSI'] = rsi_data['RSI'].astype(np.float64)

In [ ]:
rsi_data.head()

rsi_data.index

Index(['2001-08-09', '2001-08-10', '2001-08-13', '2001-08-14', '2001-08-15',
       '2001-08-16', '2001-08-17', '2001-08-20', '2001-08-21', '2001-08-22',
       ...
       '2022-04-28', '2022-04-29', '2022-05-02', '2022-05-03', '2022-05-04',
       '2022-05-05', '2022-05-06', '2022-05-09', '2022-05-10', '2022-05-11'],
      dtype='object', length=5222)

In [ ]:
print(data.shape)
# bata=data.merge(rsi_data,how='inner',left_index=True,right_index=True)
bata = data.join(rsi_data)
bata.head()

(1099, 2)


,Close,Volume,RSI
Date,,,
2018-01-02,67.989998,944500,68.7647
2018-01-03,69.449997,2124600,82.3478
2018-01-04,70.470001,2026900,87.2063
2018-01-05,69.940002,795500,73.9808
2018-01-08,68.930000,1041200,54.3473


In [ ]:
# Confirm the Testing Set length 
test_length = data[(data.index >= '2021-09-01')].shape[0]
test_length

176

In [ ]:
def CreateFeatures_and_Targets(data, feature_length):
    X = []
    Y = []

    for i in tnrange(len(data) - feature_length): 
        X.append(data.iloc[i : i + feature_length,:].values)
        Y.append(data["Close"].values[i+feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X , Y

In [ ]:
X , Y = CreateFeatures_and_Targets(data , 32)

  0%|          | 0/1067 [00:00<?, ?it/s]

In [ ]:
X.shape , Y.shape

((1067, 32, 2), (1067,))

In [ ]:
Xtrain , Xtest , Ytrain , Ytest = X[:-test_length] , X[-test_length:] , Y[:-test_length] , Y[-test_length:]

In [ ]:
Xtrain.shape , Ytrain.shape

((891, 32, 2), (891,))

In [ ]:
Xtest.shape , Ytest.shape

((176, 32, 2), (176,))

In [ ]:
class MultiDimensionScaler():
    def __init__(self):
        self.scalers = []

    def fit_transform(self , X):
        total_dims = X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:, :, i] = Scaler.fit_transform(X[:,:,i])
            self.scalers.append(Scaler)
        return X

    def transform(self , X):
        for i in range(X.shape[2]):
            X[:, :, i] = self.scalers[i].transform(X[:,:,i])
        return X 

In [ ]:
Feature_Scaler = MultiDimensionScaler()
Xtrain = Feature_Scaler.fit_transform(Xtrain)
Xtest = Feature_Scaler.transform(Xtest)

In [ ]:
Target_Scaler = MinMaxScaler()
Ytrain = Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest = Target_Scaler.transform(Ytest.reshape(-1,1))

In [ ]:
def save_object(obj , name : str):
    pickle_out = open(f"{name}.pck","wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

def load_object(name : str):
    pickle_in = open(f"{name}.pck","rb")
    data = pickle.load(pickle_in)
    return data

In [ ]:
save_object(Feature_Scaler , "Feature_Scaler")
save_object(Target_Scaler , "Target_Scaler")

Model Building

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=5, min_lr=0.00001,verbose = 1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , Bidirectional , BatchNormalization

model = Sequential()

model.add(Bidirectional(LSTM(512 ,return_sequences=True , recurrent_dropout=0.1, input_shape=(32, 3))))
model.add(LSTM(256 ,recurrent_dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64 , activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(32 , activation='elu'))
model.add(Dense(1 , activation='linear'))

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002)
model.compile(loss='mse', optimizer=optimizer)

In [ ]:
history = model.fit(Xtrain, Ytrain,
            epochs=10,
            batch_size = 10,
            verbose=1,
            shuffle=False ,
            validation_data=(Xtest , Ytest),
            callbacks=[reduce_lr , save_best])

Epoch 1/10
90/90 [==============================] - 34s 315ms/step - loss: 0.0614 - val_loss: 0.1579 - lr: 0.0020
Epoch 2/10
90/90 [==============================] - 29s 322ms/step - loss: 0.0262 - val_loss: 0.1346 - lr: 0.0020
Epoch 3/10
90/90 [==============================] - 28s 307ms/step - loss: 0.0241 - val_loss: 0.1215 - lr: 0.0020
Epoch 4/10
90/90 [==============================] - 28s 308ms/step - loss: 0.0195 - val_loss: 0.1058 - lr: 0.0020
Epoch 5/10
90/90 [==============================] - 28s 310ms/step - loss: 0.0178 - val_loss: 0.0987 - lr: 0.0020
Epoch 6/10
90/90 [==============================] - 28s 309ms/step - loss: 0.0159 - val_loss: 0.0808 - lr: 0.0020
Epoch 7/10
90/90 [==============================] - 28s 310ms/step - loss: 0.0142 - val_loss: 0.0727 - lr: 0.0020
Epoch 8/10
90/90 [==============================] - 28s 309ms/step - loss: 0.0121 - val_loss: 0.0781 - lr: 0.0020
Epoch 9/10
90/90 [==============================] - 28s 313ms/step - loss: 0.0112 - val_

In [ ]:
model.load_weights("best_weights.h5")

Visualize prediction on Test Set

In [ ]:
Predictions = model.predict(Xtest)

In [ ]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Ytest)

In [ ]:
Predictions.shape

(176, 1)

In [ ]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [ ]:

fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index[-test_length:] , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index[-test_length:] , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

Visualize Prediction on whole data

In [ ]:
Total_features = np.concatenate((Xtrain , Xtest) , axis = 0)

In [ ]:
Total_Targets = np.concatenate((Ytrain , Ytest) , axis = 0)

In [ ]:
Predictions = model.predict(Total_features)

In [ ]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Total_Targets)

In [ ]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)
Predictions.shape

(1067,)

In [ ]:
metric = tf.keras.metrics.Accuracy()
metric.update_state(Actual,Predictions)


<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1067.0>

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

In [ ]:
model.save("Model.h5")
loaded_model = tf.keras.models.load_model("Model.h5")

In [ ]:
import math


 
MSE = np.square(np.subtract(Actual.tolist(),Predictions.tolist())).mean() 
 
RMSE = math.sqrt(MSE)
RMSE

8.448500779827567

Realtime Prediction

In [ ]:
def PredictStockPrice(Model , DataFrame , PreviousDate , feature_length = 32):
    idx_location = DataFrame.index.get_loc(PreviousDate)
    Features = DataFrame.iloc[idx_location - feature_length : idx_location,:].values
    Features = np.expand_dims(Features , axis = 0)
    Features = Feature_Scaler.transform(Features)
    Prediction = Model.predict(Features)
    Prediction = Target_Scaler.inverse_transform(Prediction)
    return Prediction[0][0]

In [ ]:
PredictStockPrice(loaded_model , data , '2021-01-14')

68.27816